In [ ]:
!pip install emoji
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv('/content/Train.csv')
data = data.drop("Unnamed: 0",axis = 1)
data.head(3500)

,TEXT,Label
0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,Been friends since 7th grade. Look at us now w...,2
3,This is what it looks like when someone loves ...,3
4,RT @user this white family was invited to a Bl...,3
...,...,...
3495,Wanting To Be Someone Else Is A Waste Of Who Y...,9
3496,About last night #purposetour #confidentnature...,17
3497,Book your Keratin Lash Lift &amp; Tint TODAY n...,2
3498,** COMPETITION**Inov8 want to see YOUR best o...,6


In [ ]:
x = data["TEXT"]
y = data["Label"]

In [ ]:
x[0]

'Vacation wasted ! #vacation2017 #photobomb #tired #vacationwasted #mcgar30 #miami @ Port of…\n'

In [ ]:
x = np.asarray(x)
y = np.asarray(y)

'''
X = data[0].values
Y = data[1].values
'''

'\nX = data[0].values\nY = data[1].values\n'

In [ ]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

**map each word in data to unique num**

In [ ]:
word2index = tokenizer.word_index
word2index

{'the': 1,
 'user': 2,
 'my': 3,
 'to': 4,
 'a': 5,
 'i': 6,
 '️': 7,
 'and': 8,
 'in': 9,
 'you': 10,
 'of': 11,
 'for': 12,
 'with': 13,
 'this': 14,
 'love': 15,
 'is': 16,
 'at': 17,
 '…': 18,
 'new': 19,
 'on': 20,
 'me': 21,
 'so': 22,
 'amp': 23,
 'it': 24,
 'happy': 25,
 'we': 26,
 'day': 27,
 'be': 28,
 'was': 29,
 'all': 30,
 'your': 31,
 'from': 32,
 'that': 33,
 'out': 34,
 'just': 35,
 'our': 36,
 'are': 37,
 'one': 38,
 'night': 39,
 'york': 40,
 'time': 41,
 'beach': 42,
 'today': 43,
 'when': 44,
 'by': 45,
 'have': 46,
 'good': 47,
 'up': 48,
 'park': 49,
 'christmas': 50,
 'best': 51,
 'but': 52,
 'like': 53,
 "i'm": 54,
 'california': 55,
 'get': 56,
 "it's": 57,
 'these': 58,
 'birthday': 59,
 'last': 60,
 'got': 61,
 'city': 62,
 'beautiful': 63,
 'great': 64,
 'what': 65,
 'little': 66,
 'see': 67,
 'family': 68,
 'university': 69,
 'thanks': 70,
 'back': 71,
 'thank': 72,
 'some': 73,
 'life': 74,
 'not': 75,
 'had': 76,
 'fun': 77,
 'much': 78,
 'here': 79,
 'fa

**turn each stetment to his mapping num**

In [ ]:
Xtokens = tokenizer.texts_to_sequences(x)
Xtokens[0:2]

[[654, 6433, 16066, 5441, 1772, 24592, 24593, 150, 1418, 313],
 [213, 1363, 1191, 22, 632, 2, 1363, 199, 24594, 1364, 509, 10235]]

In [ ]:
maxlen = get_maxlen(Xtokens)
print(maxlen)

36


In [ ]:
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [ ]:
print(Xtrain[0])
print(Xtrain[0].shape)
print(Xtrain.shape)

[  654  6433 16066  5441  1772 24592 24593   150  1418   313     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
(36,)
(36,)


In [ ]:
Ytrain = to_categorical(y)
print(Ytrain[0])
print(Ytrain[0].shape)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(20,)


In [ ]:
num_class = len(Ytrain[0])
embed_size = 50
num_vocablry = len(word2index) + 1

print(num_class)
print(num_vocablry)

20
85531


**Embedding layer take**
*********************************
1 - input dim should equal len(word2index)
***********************************
2 - output dim = num of feature you need each word to have
***********************************
3 - input lentgh = what is the max lentgh of any stetment
*******************************************
4 - output shape of this layer = max lentgh*num of feature for each word
********************************************************
**mean of return seq = true**
Return sequences refer to return the hidden state a<t>. By default, the return_sequences is set to False in Keras RNN layers, and this means the RNN layer will only return the last hidden state output a<T>. The last hidden state output captures an abstract representation of the input sequence. In some case, it is all we need, such as a classification or regression model where the RNN is followed by the Dense layer(s) to generate logits for news topic classification or score for sentiment analysis, or in a generative model to produce the softmax probabilities for the next possible char.

In other cases, we need the full sequence as the output. Setting return_sequences to True is necessary


In [ ]:
model = Sequential([
    Embedding(input_dim = len(word2index) + 1, output_dim = embed_size , input_length=maxlen),

    LSTM(units = 16, return_sequences = True),
    LSTM(units = 4),
    Dense(num_class, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 36, 50)            4276550   
                                                                 
 lstm_2 (LSTM)               (None, 36, 16)            4288      
                                                                 
 lstm_3 (LSTM)               (None, 4)                 336       
                                                                 
 dense_1 (Dense)             (None, 20)                100       
                                                                 
Total params: 4281274 (16.33 MB)
Trainable params: 4281274 (16.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(Xtrain, Ytrain, epochs = 50, verbose=1)

Epoch 1/50
2188/2188 [==============================] - 45s 18ms/step - loss: 2.6659 - accuracy: 0.2192
Epoch 2/50
2188/2188 [==============================] - 21s 10ms/step - loss: 2.4176 - accuracy: 0.2703
Epoch 3/50
2188/2188 [==============================] - 20s 9ms/step - loss: 2.2035 - accuracy: 0.3400
Epoch 4/50
2188/2188 [==============================] - 20s 9ms/step - loss: 1.9963 - accuracy: 0.4006
Epoch 5/50
2188/2188 [==============================] - 19s 9ms/step - loss: 1.8139 - accuracy: 0.4436
Epoch 6/50
2188/2188 [==============================] - 20s 9ms/step - loss: 1.6645 - accuracy: 0.4749
Epoch 7/50
2188/2188 [==============================] - 19s 8ms/step - loss: 1.5401 - accuracy: 0.5005
Epoch 8/50
2188/2188 [==============================] - 19s 9ms/step - loss: 1.4288 - accuracy: 0.5375
Epoch 9/50
2188/2188 [==============================] - 19s 9ms/step - loss: 1.3304 - accuracy: 0.5802
Epoch 10/50
2188/2188 [==============================] - 22s 10ms/step 

In [ ]:
emoji = pd.read_csv('/content/Mapping.csv')
emoji = emoji.drop("Unnamed: 0",axis = 1)
emoji.head(20)

,emoticons,number
0,😜,0
1,📸,1
2,😍,2
3,😂,3
4,😉,4
5,🎄,5
6,📷,6
7,🔥,7
8,😘,8
9,❤,9


In [ ]:
emoji_dict = emoji.to_dict()
emoji_dict = emoji_dict['emoticons']
emoji_dict

{0: '😜',
 1: '📸',
 2: '😍',
 3: '😂',
 4: '😉',
 5: '🎄',
 6: '📷',
 7: '🔥',
 8: '😘',
 9: '❤',
 10: '😁',
 11: '🇺🇸',
 12: '☀',
 13: '✨',
 14: '💙',
 15: '💕',
 16: '😎',
 17: '😊',
 18: '💜',
 19: '💯'}

In [ ]:
def label_to_emoji(label):
    return emoji_dict[label]

x= label_to_emoji(2)
x

'😍'

In [ ]:
test = pd.read_csv('/content/Test.csv')
test = test.drop("Unnamed: 0",axis = 1)
test.head(2)

,id,TEXT
0,0,Thought this was cool...#Repost (get_repost)・・...
1,1,Happy 4th! Corte madera parade. #everytownusa ...


In [ ]:
test = test['TEXT'].values.tolist()
test[0]

'Thought this was cool...#Repost (get_repost)・・・Colorview. by shay_images…\n'

In [ ]:
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')
Xtest

In [ ]:
y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

812/812 [==============================] - 4s 5ms/step


In [ ]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

Streaming output truncated to the last 5000 lines.
@user This Cubs fan will always say Hi to you #Cubs #Bears @user
 😍
#LasVegasStrip at #sunrise with #Jameson in hand. #Goals haha So…
 😁
I love this man for all that he does for his family ️ I was so lucky…
 ❤
Skinny Pete always has a good time fishing #fridaynightfishing @ Lynnhaven Fishing Pier
 📸
I had someone special come see me at the salon today that touched my heart so deeply, &amp; I…
 ❤
Happy birthday to my girl Emily ! Hope 16 treats you well @user
 💕
My handsome guy @ New Haven County, Connecticut
 😍
This is what life is like after your church finishes a 21 Day fast! #Joy #IceCream #Fentons…
 😂
SoCal found me too! : @user #edc20 #edc2016 #edc16 #edclv #tbt #eletricdaisycarnival…
 😁
i had an awesome time at #longbeachroxxhuge thanks to @user for making this possible @ Long…
 😍
Where dreams are made of @ New York, New York
 🔥
What a beautiful Sunset @ City of Inglewood
 ❤
The one and only, happy birthday my king @user #Michael

# comments

In [ ]:
"""from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
import keras
k_init = keras.initializers.Constant(value=0.5)
b_init = keras.initializers.Constant(value=0)
r_init = keras.initializers.Constant(value=0.5)
# LSTM units
units = 1

# define model
inputs1 = Input(shape=(4, 2))
lstm1 = LSTM(units, activation='relu',return_sequences=True, kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = array([1, 1, 1, 1, 1, 1,1,1]).reshape((1,4,2))
# make and show prediction
output = model.predict(data)
print(output, output.shape)"""

"from keras.models import Model\nfrom keras.layers import Input\nfrom keras.layers import LSTM\nfrom numpy import array\nimport keras\nk_init = keras.initializers.Constant(value=0.5)\nb_init = keras.initializers.Constant(value=0)\nr_init = keras.initializers.Constant(value=0.5)\n# LSTM units\nunits = 1\n\n# define model\ninputs1 = Input(shape=(4, 2))\nlstm1 = LSTM(units, activation='relu',return_sequences=True, kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init)(inputs1)\nmodel = Model(inputs=inputs1, outputs=lstm1)\n# define input data\ndata = array([1, 1, 1, 1, 1, 1,1,1]).reshape((1,4,2))\n# make and show prediction\noutput = model.predict(data)\nprint(output, output.shape)"